In [0]:
#requirements
!pip install reverse_geocoder 

# Data Integration

In [0]:
from data.data_join import *
join_europe_by_country_and_save(
    out_countries_dir="dbfs:/vibebnb/data/europe_countries_joined",
)

#Scoring

In [0]:
from score import *
t_job = time.perf_counter()
print("[SCORING] Job started")

in_path = "dbfs:/vibebnb/data/europe_countries_joined"
out_path = "dbfs:/vibebnb/data/europe_countries_scored.parquet"

t0 = time.perf_counter()
df = spark.read.parquet(in_path).persist(StorageLevel.MEMORY_AND_DISK)
count = df.count()
print(f"[SCORING] Loaded {count:,} rows in {time.perf_counter() - t0:.2f}s")

df_scored = scoring_all(df)

t0 = time.perf_counter()
(
    df_scored
    .write
    .mode("overwrite")
    .partitionBy("addr_cc")
    .parquet(out_path)
)
print(f"[SCORING] Saved output in {time.perf_counter() - t0:.2f}s")

df.unpersist()
print(f"[SCORING] Job finished in {time.perf_counter() - t_job:.2f}s")

# Embedding and LSH Index

In [0]:
from embeddings_fit import *
build_global_models_and_save(
    scored_input_path="dbfs:/vibebnb/data/europe_countries_scored.parquet",
    embedded_out_path="dbfs:/vibebnb/data/europe_countries_embedded",
    emb_model_path="dbfs:/vibebnb/models/embedding_pipeline_global",
    lsh_model_path="dbfs:/vibebnb/models/lsh_global"
)
